In [ ]:
import pandas as pd
import numpy as np

# Carregar o arquivo
df = pd.read_excel("data/raw/Online Retail.xlsx")

# Visualizar estrutura
print(df.shape)  # Número de linhas e colunas
print(df.info()) # Tipos de dados e valores nulos
df.head()        # Primeiras linhas



#  Estatísticas Descritivas Básicas
# Resumo estatístico de colunas numéricas
df.describe()

# Medidas específicas
print("Média de quantidade:", df['Quantity'].mean())
print("Mediana de preço:", df['UnitPrice'].median())
print("Desvio padrão de quantidade:", df['Quantity'].std())

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/Online Retail.xls'

In [ ]:
# Limpeza de Dados (Tratamento de Valores Nulos/Outliers)
# Verificar nulos
print(df.isnull().sum())

# Remover linhas com CustomerID nulo
df_clean = df.dropna(subset=['CustomerID'])

# Filtrar quantidades e preços positivos (evitar devoluções)
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['UnitPrice'] > 0]

In [ ]:
# Análise Exploratória de Dados (EDA) 
# Análise de Distribuições e Tendências
# Histograma de quantidades
df_clean['Quantity'].hist(bins=50)

# Boxplot para detectar outliers
df_clean[['Quantity', 'UnitPrice']].plot(kind='box')

# Correlação entre variáveis numéricas
df_clean[['Quantity', 'UnitPrice']].corr()

In [ ]:
# Vendas totais por país
#Agregações com GroupBy

sales_by_country = df_clean.groupby('Country')['Quantity'].sum().sort_values(ascending=False)

# Faturamento por produto (Quantity * UnitPrice)
df_clean['TotalRevenue'] = df_clean['Quantity'] * df_clean['UnitPrice']
revenue_by_product = df_clean.groupby('StockCode')['TotalRevenue'].sum().sort_values(ascending=False)

In [ ]:
# Séries Temporais
# Converter InvoiceDate para datetime

df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Vendas diárias
daily_sales = df_clean.set_index('InvoiceDate').resample('D')['TotalRevenue'].sum()

# Plotar série temporal
daily_sales.plot(title='Vendas Diárias')

In [ ]:
#  Análise de Cesta de Compras (Associação)

from mlxtend.frequent_patterns import apriori, association_rules

# Criar matriz de cesta de compas por invoice
basket = df_clean.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
basket_binary = basket.applymap(lambda x: 1 if x > 0 else 0)

# Aplicar algoritmo Apriori
frequent_itemsets = apriori(basket_binary, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
# ESTATÍSTICAS POR  CLIENTE (RFM)

# Recência, Frequência, Valor Monetário (RFM)
snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recência
    'InvoiceNo': 'nunique',                                  # Frequência
    'TotalRevenue': 'sum'                                    # Valor Monetário
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalRevenue': 'Monetary'})